In [1]:
import pandas as pd
import numpy as np
from scipy.interpolate import CubicSpline
import pickle

In [2]:
all_dfs = pickle.load(open('data/all_dfs.pkl', 'rb'))

In [3]:
all_precincts = ['1', '5', '6', '7', '9', '10', '13', '14', '17', '18', '19', '20', '22',
       '23', '24', '25', '26', '28', '30', '32', '33', '34', '40', '41', '42',
       '43', '44', '45', '46', '47', '48', '49', '50', '52', '60', '61', '62',
       '63', '66', '67', '68', '69', '70', '71', '72', '73', '75', '76', '77',
       '78', '79', '81', '83', '84', '88', '90', '94', '100', '101', '102',
       '103', '104', '105', '106', '107', '108', '109', '110', '111', '112',
       '113', '114', '115', '120', '121', '122', '123']

In [4]:
bike_counts = np.zeros(shape=len(all_precincts), dtype='int64')

for key in list(all_dfs.keys()):
    all_dfs[key] = all_dfs[key].loc[np.all(all_dfs[key].notna(), axis=1)]
    unique_precincts, counts = np.unique(all_dfs[key], return_counts=True)
    counts = counts[np.argsort(list(map(int, list(unique_precincts))))]

    diff = list(map(int, list(set(all_precincts) - set(unique_precincts))))
    diff.sort()
    diff = list(map(str, diff))

    for precinct in diff:
        idx = all_precincts.index(precinct)
        counts = np.insert(counts, idx, 0)
    
    bike_counts = np.vstack([bike_counts, counts/2])

bike_counts = bike_counts[1:]

In [5]:
df = pd.DataFrame(bike_counts, columns=all_precincts)

In [6]:
keys = ['201' + str(Y) + '-' + str(M).rjust(2, '0') for Y in range(4, 10) for M in range(1, 13)]
df.index = list(all_dfs.keys())
diff = list(set(keys) - set(all_dfs.keys()))
diff.sort()

In [7]:
for key in diff:
    aux = np.empty(shape=len(all_precincts))
    aux[:] = np.NaN
    df.loc[key] = aux.copy()
    
df = df.sort_index()
df.index = pd.DatetimeIndex(df.index)

In [8]:
df.head()

,1,5,6,7,9,10,13,14,17,18,...,110,111,112,113,114,115,120,121,122,123
2014-01-01,34782.5,17553.5,29745.0,10819.5,29580.5,28003.0,36118.5,44401.0,19588.0,25516.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-02-01,25988.5,14142.5,24019.0,8092.0,23946.5,21520.5,27951.5,30917.0,13964.0,16159.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-03-01,52251.0,26549.5,44865.0,15217.5,42675.0,41449.0,53198.0,62522.0,27272.5,35970.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-04-01,85821.0,40936.0,73288.5,20873.5,63335.0,64474.0,74846.5,90386.0,41948.0,55582.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-05-01,120570.0,55464.5,92629.0,28127.5,79449.5,82365.0,97697.5,109153.0,51942.5,67909.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
for column in df.columns:
    df.loc[:, column] = df.loc[:, column].interpolate(method='polynomial', order=5)

In [10]:
np.any(df.isna())

False